# Análisis de Datos (III): extrayendo vocabulario para clasificar textos

En esta práctica vamos a poner en práctica todo lo aprendido con pandas, numpy y hasta con expresiones regulares. El objetivo de la práctica es tratar de extraer un conjunto de palabras (vocabulario) que nos permitan clasificar (de una manera muy sencilla) si una crítica o review de una película es positiva o negativa sin siquiera leer el texto.

Para ello, os proponemos un conjunto de 5000 críticas de películas del portal imdb. Cada una de estas críticas está previamente catalogada como positiva (sentiment = 1) o negativa (sentiment = 0). El hecho de que tengamos conocimiento de qué tipo de crítica es (positiva o negativa) nos va a permitir crear un pequeño y simple clasificador de críticas que luego podremos evaluar. 

Los pasos que vamos a dar son los siguientes:
- Importar el conjunto de críticas en un DataFrame. 
- "Limpiar" los comentarios para tratar de dejar todos ellos en un mismo formato: todo en minúsculas, sin caracteres especiales, etc.
- Separar el conjunto de críticas en un conjunto de entrenamiento (alrededor del 80% de las muestras) y un conjunto de test (alrededor del 20% de las muestras)
- Extrar dos diccionarios de palabras: un diccionario de palabras "positivas" y un diccionario de palabras "negativas". A partir de estos diccionarios crearemos un DataFrame vocabulario que nos ayudará a clasificar nuevas muestras
- A partir de un comentario de test, extraeremos el diccionario de dicho comentario, lo "combinaremos" con el vocabulario y veremos si tiene más palabras "positivas" o "negativas". En función del número de palabras lo clasificaremos como comentario positivo o comentario negativo

In [1]:
import pandas as pd
import re
import numpy as np

A continuación vamos a crar el DataFrame comentarios. Echa un vistazo al número de elementos y columnas que tiene, cuáles son las 5 primeras filas, etc.

In [2]:
df = pd.read_csv('comentarios.tsv', sep='\t')

In [3]:
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


Cuenta el número de comentarios positivos y negativos. ¿Podemos decir que el número de críticas está "balanceado" (hay un número parecido de comentraios positivos y negativos)?

In [4]:
df[df.sentiment==0].id.count()

2483

In [5]:
df[df.sentiment==1].id.count()

2517

### Limpieza de datos

Vamos a comenzar con una pequeña limpieza de los datos para crear un conjunto de palabras lo suficientemente bueno. Principalmente vamos a:
- eliminar cualquier cosa que no sea una letra mayúscula o minúscula (sustituiremos cada caracter especial por un espacio en blanco
- pasar todo a minúsculas
- dividir el texto en una lista de palabras
- eliminar aquellas palabras de longitud  cero
- eliminar aquellas palabras que forman parte de la lista de stop_words: palabras que, por la cantidad de veces que se usan, no aportan nada de informacion

Crea una función procesa_comentario que reciba como parámetro de entrada un comentario y devuelva un listado de palabras "limpias".

- Para eliminar cualquier cosa que no sea una letra, utiliza la función re.sub. Esta función reciba 3 parámetros de entrada: la expresión regular que tiene que buscar para eliminar; la cadena de caracteres por las que va a eliminar cada match de la expresión regular; y el string donde va a trabajar la función. 
- Para pasar todo a minúsculas existe la función lower
- Para dividir un texto en un listado de palabras existe la función split
- Para eliminar palabras de una lista puedes utilizar la función filter

In [6]:
expresion_regular = re.compile('[^a-zA-Z]')

In [7]:
lista_stop = ['a','about','above','after','again','against','all','am','an','and','any','are','arent','as','at','be','because','been','before','being',
'below','between','both','but','by','cant','cannot','could','couldnt','did','didnt','do','does','doesnt','doing','dont','down','during','each',
'few','for','from','further','had','hadnt','has','hasnt','have','havent','having','he','hed','hell','hes','her','here',
'heres','hers','herself','him','himself','his','how','hows','i','id','ill','im','ive','if','in','into','is','isnt','it',
'its','its','itself','lets','me','more','most','mustnt','my','myself','no','nor','not','of','off','on','once','only','or',
'other','ought','our','ours','ourselves','out','over','own','same','shant','she','shed','shell','shes','should','shouldnt',
'so','some','such','than','that','thats','the','their','theirs','them','themselves','then','there','theres','these','they','theyd',
'theyll','theyre','theyve','this','thos','through','to','too','under','until','up','very','was','wasnt','we','wed','well',
'were','weve','were','werent','what','whats','when','whens','where','wheres','which','while','who','whos','whom','why',
'whys','with','wont','would','wouldnt','you','youd','youll','youre','youve','your','yours','yourself','yourselves']

In [8]:
def procesa_comentario(comentario):
    comentario = re.sub(expresion_regular, ' ', comentario)
    comentario = comentario.lower()
    comentario = comentario.split(" ")
    comentario = filter(lambda x: len(x)>0, comentario)
    comentario = filter(lambda x: x not in lista_stop, comentario)
    return comentario

Modifica el DataFrame comentarios de forma que la columna review deje de ser un texto para convertirse en un listado de palabras. ¿Puedes hacer esto sin estructuras iterativas? Piensa en la función apply

In [9]:
comentarios = df
comentarios['review'] = comentarios['review'].apply(procesa_comentario)
comentarios

,id,sentiment,review
0,5814_8,1,<filter object at 0x0000017FF5A19548>
1,2381_9,1,<filter object at 0x0000017FF5A1EC48>
2,7759_3,0,<filter object at 0x0000017FF5A27A48>
3,3630_4,0,<filter object at 0x0000017FF5A2D788>
4,9495_8,1,<filter object at 0x0000017FF5A33408>
...,...,...,...
4995,3720_2,0,<filter object at 0x0000017FFB3CD5C8>
4996,4229_10,1,<filter object at 0x0000017FFB3D06C8>
4997,8042_3,0,<filter object at 0x0000017FFB3D2808>
4998,9669_9,1,<filter object at 0x0000017FFB3D3F08>


### División del DataFrame en train y set

Vamos a dividir nuestra DataFrame en dos DataFrames de entrenamiento (4000 comentarios) y test (1000 comentarios). Para ello, vamos a seleccionar 4000 índices aleatorios entre 1 y 5000. Para esto, vamos a utilizar la función np.random.choice, que elige 4000 números entre 0 y 4999 sin repetición. A partir de dicho conjunto de índices, crea el conjunto índices test y divide el DataFrame comentarios en comentarios_train y comentarios_test

In [10]:
np.random.seed(1234)
indices_train =  np.random.choice(5000, 4000, replace = False)
indices_test = [x for x in np.arange(5000) if x not in indices_train]
comentarios_train = comentarios.iloc[indices_train,:]
comentarios_test = comentarios.iloc[indices_test,:]

### Creación de diccionarios

Vamos a crear dos diccionarios: diccionario_negativo y diccionario_positivo. Cada diccionario se creará a partir de los comentarios negativos y positivos de entrenamiento, respectivamente. El diccionario deberá contener el conjunto de palabras que aparecen en todos los comentarios (como clave) y el número de veces que aparecen (como valor). 

In [11]:
diccionario_negativo = {}
diccionario_positivo = {}
for indice, fila in comentarios_train.iterrows():
    for palabra in fila['review']:
        if fila['sentiment'] == 0:
            diccionario_negativo[palabra] = diccionario_negativo.get(palabra, 0) + 1
        else:
            diccionario_positivo[palabra] = diccionario_positivo.get(palabra, 0) + 1

Una vez construidos los diccionarios vamos a convertilos cada uno de ellos en una Series. combinarlos en un DataFrame de tantas filas como palabras aparezcan en el conjunto de diccionarios y dos columnas: 'negativo' y 'positivo' que contienen el número de veces que dichas palabras aparecen en cada tipo de comentarios respectivamente. A este DataFrame le llamaremos vocabuario

In [12]:
serie_negativo = pd.Series(diccionario_negativo, name='negativo')
#serie_negativo = serie_negativo.fillna(0)
serie_positivo = pd.Series(diccionario_positivo, name='positivo')
#serie_positivo = serie_positivo.fillna(0)
serie_positivo.sort_values(ascending=False)

br           7994
s            5611
film         3441
movie        3087
t            2195
             ... 
dorday          1
tooltime        1
fuelling        1
splashed        1
wikipedia       1
Name: positivo, Length: 25511, dtype: int64

Analiza las dos series utilizando la función describe. Observa que, por ejemplo, serie_negativo contiene 24193 palabras y que la media de veces que dicha palabra aparece es 10.72. Sin embargo, la desviación típica es muy alta y el valor de cuartil 3 es muy bajo: esto indica que hay muchas palabras que aparecen muy pocas veces. Lo mismo ocurre con serie_positivo.

Para reducir el número de apariciones, vamos a eliminar de cada serie aquellas palabras que no aparezcan tantas veces como la media actual de apariciones. Observa que ahora nos quedamos con únciamente 3334  palabras "negtivas" y 3718 "positivas"

In [14]:
print (serie_negativo.describe())
print (serie_positivo.describe())

count    24193.000000
mean        10.720746
std         87.631764
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max       8672.000000
Name: negativo, dtype: float64
count    25511.000000
mean        10.630003
std         80.975071
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max       7994.000000
Name: positivo, dtype: float64


In [15]:
serie_positivo = serie_positivo[serie_positivo>serie_positivo.mean()]
serie_negativo = serie_negativo[serie_negativo>serie_negativo.mean()]

In [16]:
print (serie_negativo.describe())
print (serie_positivo.describe())

count    3334.000000
mean       62.474205
std       229.352627
min        11.000000
25%        15.000000
50%        23.000000
75%        49.000000
max      8672.000000
Name: negativo, dtype: float64
count    3718.000000
mean       58.263852
std       205.707662
min        11.000000
25%        15.000000
50%        23.000000
75%        47.000000
max      7994.000000
Name: positivo, dtype: float64


In [17]:
vocabulario = pd.concat([serie_negativo, serie_positivo], axis = 1, sort=False)
vocabulario = vocabulario.fillna(0)
vocabulario

,negativo,positivo
first,697.0,751.0
m,531.0,307.0
dog,73.0,62.0
movie,3803.0,3087.0
find,281.0,412.0
...,...,...
amrita,0.0,17.0
dric,0.0,15.0
gandhi,0.0,11.0
lincoln,0.0,14.0


Muestra por pantalla las 10 palabras "negativas" que más veces aparecen y las 10 palabras "positivas" que más veces aparecen

In [19]:
print (vocabulario['negativo'].sort_values(ascending=False).head(10))
print (vocabulario['positivo'].sort_values(ascending=False).head(10))

br       8672.0
s        5242.0
movie    3803.0
t        3230.0
film     3154.0
one      2082.0
like     1862.0
just     1644.0
can      1270.0
even     1247.0
Name: negativo, dtype: float64
br       7994.0
s        5611.0
film     3441.0
movie    3087.0
t        2195.0
one      2160.0
like     1487.0
good     1300.0
just     1237.0
can      1105.0
Name: positivo, dtype: float64


Vamos a transformar la información cuantitativa (cuántas veces aparece cada palabra) en un porcentaje relativo. Es decir, dividir cada fila del DataFrame entre la suma del número de veces que dicha palabra aparece en comentarios "positivos" y "negativos". De esta manera tenemos algo parecido a la "probabilidad" de que dicha palabra aparezca en un comentario "positivo" en uno "negativo". Puedes utilizar la función apply

In [20]:
vocabulario = vocabulario.apply(lambda x: x/x.sum(),axis=1)
vocabulario.fillna(0)
vocabulario

,negativo,positivo
first,0.481354,0.518646
m,0.633652,0.366348
dog,0.540741,0.459259
movie,0.551959,0.448041
find,0.405483,0.594517
...,...,...
amrita,0.000000,1.000000
dric,0.000000,1.000000
gandhi,0.000000,1.000000
lincoln,0.000000,1.000000


### Clasificación de nuevos comentarios

A partir del DataFrame comentarios_test vamos a hacer lo siguiente. Para cada comentario de test:
- obtener diccionario_test con las palabras de dicho comentario junto con el número de veces que aparece
- transformar dicho diccionario en una Series
- multiplicar la serie_test por cada columna del DataFrame. De esta manera, cada palabra que aparezca en serie_test se multiplicará por la probabilidad de ser positivo o negativo. 
- Sumar por columnas el resultado de la multiplicación anterior: esto nos dará un resultado final de probabilidades acumulada
- Obtener el nombre de la columna que mayor suma ha obtenido: si es la columna "negativo" entonces predecimos que dicho comentario es de tipo sentiment=0; al contrario predeciremos sentiment=1;
- Contar el número de acierots/fallos de nuestro clasificador

In [21]:

num_aciertos = 0
num_fallos = 0
for indice, fila in comentarios_test.iterrows():
    diccionario_test = {}
    for palabra in fila['review']:
        diccionario_test[palabra] = diccionario_test.get(palabra, 0) + 1
    serie_test = pd.Series(diccionario_test)
    prediccion = vocabulario.multiply(serie_test, axis=0).sum(axis = 0).idxmax()
    if prediccion=="positivo" and fila['sentiment'] ==1:
        num_aciertos +=1
    elif prediccion=="negativo" and fila['sentiment'] ==0:
        num_aciertos +=1
    else:
        num_fallos += 1

In [22]:
print (num_aciertos)
print (num_fallos)

798
202
